**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**July, 2023**

---

# Data integration of placenta infection at 48hpi using scVI


In [1]:
from __future__ import print_function
import torch

import sys, os
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True'
sys.path.insert(1, './')

In [2]:
# Seed for reproducibility
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple

# scVI imports
import scvi
from scvi.dataset import AnnDatasetFromAnnData
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)



def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '../results/scVI/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.3.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_ser

'/opt/conda/envs/scvi-singularity/bin/python'

In [3]:
def identityTransfer(adata_CellsNewID, adata, field='cell_type'):
    
    '''
    Function to transfer the value in a .obs column, modifying only the cells included in adata_CellsNewID
    
    Parameters:
        adata_CellsNewID: anndata object with cells. These should have in their .obs[field] the new values to be transfer to adata
        adata: anndata object to modify. The value in .obs[field] will be modifies for all the cells in adata_CellsNewID
        field: column name in .obs to operate in.
        
     Return:
        anndata object with the metadata modified
        
    '''
    
    #converting categorical into strings to introduce new categories
    adata.obs[field]=adata.obs[field].astype("string")


    #assigning the new categories to the cells in adata
    for cell in adata_CellsNewID.obs.index:
        adata.obs[field][cell] = adata_CellsNewID.obs[field][cell]

    #Returning strings into categorical 
    adata.obs[field]=adata.obs[field].astype("category")
    
    return(adata)

### Loading non-normalized data

In [4]:
adata = sc.read('../data/rna1_counts_preprocessed_All_adata.h5ad')

In [5]:
adata

AnnData object with n_obs × n_vars = 119146 × 36601
    obs: 'sample', 'stage', 'donor', 'hpi', 'infection', 'percent_mito', 'n_counts', 'batch', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'is_doublet', 'infection_stage', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes'
    var: 'gene_ids', 'feature_types', 'mean-0', 'std-0', 'mean-1', 'std-1', 'mean-2', 'std-2', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'donor_id_colors', 'hvg', 'infection_colors', 'sample_colors'
    layers: 'raw_counts'

### Compute the scVI latent space

Based on the scVI documentation.

In [6]:
# do some basic preprocessing
#adata.layers["raw_counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
#adata.raw = adata.copy()

normalizing counts per cell
    finished (0:00:06)


In [7]:
np.unique(adata.obs['donor_id'])

array(['Hrv136', 'Hrv232', 'Hrv236', 'donor_doublets', 'scDonor_Tg1',
       'scDonor_Tg2', 'scDonor_Tg3', 'scDonor_Tg4'], dtype=object)

In [8]:
adata.obs['sample']

Pla_HDBR13007976_AAACCCAAGCCTCATA    Pla_HDBR13007976
Pla_HDBR13007976_AAACCCAAGCTGTTAC    Pla_HDBR13007976
Pla_HDBR13007976_AAACCCAAGGCATCAG    Pla_HDBR13007976
Pla_HDBR13007976_AAACCCAAGGGAGATA    Pla_HDBR13007976
Pla_HDBR13007976_AAACCCACACTAAACC    Pla_HDBR13007976
                                           ...       
Pla_HDBR13661576_TTTGTTGGTCACTGAT    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGGTTTCGGCG    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGTCAGCTGAT    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGTCGGTTGTA    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGTCTGGTTGA    Pla_HDBR13661576
Name: sample, Length: 119146, dtype: category
Categories (12, object): ['Pla_HDBR12330714', 'Pla_HDBR12330715', 'Pla_HDBR13007976', 'Pla_HDBR13007977', ..., 'Pla_HDBR13661575', 'Pla_HDBR13661576', 'Pla_HDBR13798225', 'Pla_HDBR13798226']

In [9]:
adata.obs

,sample,stage,donor,hpi,infection,percent_mito,n_counts,batch,sample_barcode,assignment_SoC,...,G2M_score,phase,n_genes_by_counts,total_counts,total_counts_hs,pct_counts_hs,total_counts_tg,pct_counts_tg,Tg_infected,n_genes
Pla_HDBR13007976_AAACCCAAGCCTCATA,Pla_HDBR13007976,UI_48h,Hrv171_Hrv172,48h,UI,0.035724,32751.0,0,Pla_HDBR13007976_AAACCCAAGCCTCATA,1,...,-0.026165,G1,6472.0,32751.0,32751.0,100.0,0.0,0.0,False,6472
Pla_HDBR13007976_AAACCCAAGCTGTTAC,Pla_HDBR13007976,UI_48h,Hrv171_Hrv172,48h,UI,0.023832,33232.0,0,Pla_HDBR13007976_AAACCCAAGCTGTTAC,0,...,-0.092429,G1,3822.0,33232.0,33232.0,100.0,0.0,0.0,False,3822
Pla_HDBR13007976_AAACCCAAGGCATCAG,Pla_HDBR13007976,UI_48h,Hrv171_Hrv172,48h,UI,0.017998,17502.0,0,Pla_HDBR13007976_AAACCCAAGGCATCAG,1,...,-0.057789,S,4489.0,17502.0,17502.0,100.0,0.0,0.0,False,4489
Pla_HDBR13007976_AAACCCAAGGGAGATA,Pla_HDBR13007976,UI_48h,Hrv171_Hrv172,48h,UI,0.142183,6358.0,0,Pla_HDBR13007976_AAACCCAAGGGAGATA,0,...,-0.092744,G1,450.0,6358.0,6358.0,100.0,0.0,0.0,False,450
Pla_HDBR13007976_AAACCCACACTAAACC,Pla_HDBR13007976,UI_48h,Hrv171_Hrv172,48h,UI,0.014511,29081.0,0,Pla_HDBR13007976_AAACCCACACTAAACC,1,...,0.007552,G2M,4465.0,29081.0,29081.0,100.0,0.0,0.0,False,4465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pla_HDBR13661576_TTTGTTGGTCACTGAT,Pla_HDBR13661576,Lm_48h,Hrv236,48h,Lm,0.026275,37298.0,2,Pla_HDBR13661576_TTTGTTGGTCACTGAT,not_pooled,...,-0.076960,S,NaN,NaN,NaN,NaN,NaN,NaN,nan,7275
Pla_HDBR13661576_TTTGTTGGTTTCGGCG,Pla_HDBR13661576,Lm_48h,Hrv236,48h,Lm,0.044556,6576.0,2,Pla_HDBR13661576_TTTGTTGGTTTCGGCG,not_pooled,...,1.025595,G2M,NaN,NaN,NaN,NaN,NaN,NaN,nan,3047
Pla_HDBR13661576_TTTGTTGTCAGCTGAT,Pla_HDBR13661576,Lm_48h,Hrv236,48h,Lm,0.049094,21082.0,2,Pla_HDBR13661576_TTTGTTGTCAGCTGAT,not_pooled,...,-0.028451,S,NaN,NaN,NaN,NaN,NaN,NaN,nan,5860
Pla_HDBR13661576_TTTGTTGTCGGTTGTA,Pla_HDBR13661576,Lm_48h,Hrv236,48h,Lm,0.082192,3942.0,2,Pla_HDBR13661576_TTTGTTGTCGGTTGTA,not_pooled,...,0.129952,G2M,NaN,NaN,NaN,NaN,NaN,NaN,nan,1988


In [10]:
scvi.data.setup_anndata(
    adata,
    layer="raw_counts",
    batch_key='donor_id', #samples as a batch
    #donor as a covariate of the cells
    categorical_covariate_keys=["sample"] 
    #categorical_covariate_keys=['donor_souporcell',], #
    #continuous_covariate_keys=[""]
)

INFO     Using batches from adata.obs["donor_id"]                                            


Using batches from adata.obs["donor_id"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.layers["raw_counts"]                                          


...

Computing library size prior per batch


INFO     Successfully registered anndata object containing 119146 cells, 36601 vars, 8       
         batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates   
         and 0 extra continuous covariates.                                                  


Successfully registered anndata object containing 119146 cells, 36601 vars, 8 batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates and 0 extra continuous covariates.


INFO     Please do not further modify adata until model is trained.                          


Please do not further modify adata until model is trained.


In [11]:
#---- check #layers

In [12]:
models = {}

# let's try a few values
n_latent_values = [20, 30]

for n_latent_value in n_latent_values:
    print('n_latent_value', n_latent_value)
    models[n_latent_value] = scvi.model.SCVI(adata, n_latent = n_latent_value)

n_latent_value 20
n_latent_value 30


In [13]:
models[20]

SCVI Model with the following params: 
n_hidden: 128, n_latent: 20, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb, latent_distribution: normal
Training status: Not Trained

To print summary of associated AnnData, use: scvi.data.view_anndata_setup(model.adata)

In [15]:
latent_representations = {}

for n_latent_value in n_latent_values:
    print('training model for n_latent_value:', n_latent_value)
    models[n_latent_value].train()
    
    latent_representations[n_latent_value] = models[n_latent_value].get_latent_representation()
    
    adata.obsm["X_scVI_n_latent_" + str(n_latent_value)] = latent_representations[n_latent_value]
    
    curr_df = pd.DataFrame(adata.obsm["X_scVI_n_latent_" + str(n_latent_value)])
    curr_df.to_csv('../results/scVI/20230707_obsm_with_scVI_latent_representation_n_' + str(n_latent_value) + '_allSamples48h.csv')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 20
Epoch 67/67: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 67/67 [26:02<00:00, 23.32s/it, loss=1.09e+04, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 30
Epoch 67/67: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 67/67 [25:58<00:00, 23.26s/it, loss=1.05e+04, v_num=1]


## Saving data

In [16]:
adata.X= adata.layers["raw_counts"].copy()

In [17]:
adata.write('../results/scVI/rna2_scVIintegrated_AllData.h5ad')

In [18]:
adata.var["feature_types"].value_counts()


Gene Expression    36601
Name: feature_types, dtype: int64

In [ ]:
adata